In [12]:
import datetime
import sys
import logging
import os

import geopandas as gp

sys.path.insert(0, os.path.abspath('../../'))
from Cycling_Safe.utils.osm.container import OSMContainer
import Cycling_Safe.utils.osm.osm2shp 
reload(Cycling_Safe.utils.osm.osm2shp )
from Cycling_Safe.utils.osm.osm2shp import *

from Cycling_Safe.utils.osm.osmFliter import *
from seg_nearby import get_osm_ids_near_dc_seg, bfr_20m
import Cycling_Safe.utils.geofunc
reload(Cycling_Safe.utils.geofunc)
from Cycling_Safe.utils.geofunc import merge_within
import pandas as pd
from collections import defaultdict, Counter


In [6]:

def build_gpdf_list_from_osm(osm_data, nodes, ways, rltns):
    columns = ['id', 'type', 'geometry']
    shpobjs_from_nodes = [(node.id, 'Node', node2pt(node)) for node in nodes]
    shpobjs_from_ways = [(way.id, 'Way', way2lineOrpoly(osm_data, way)) for way in ways]
    shpobjs_from_rltns = [rltn2mergedFlattenListShp(osm_data, rltn) for rltn in rltns]
    shpobjs_from_rltns = [r for r in shpobjs_from_rltns if r]
    shpobjs_from_rltns = [(rltn.id, 'Relation', shpobj) for shpobjs in shpobjs_from_rltns for shpobj in shpobjs]
    print 'build_gpdf_list len(shpobjs_from_nodes)={},len(shpobjs_from_ways)={},len(shpobjs_from_rltns)={}'.format(
        len(shpobjs_from_nodes), len(shpobjs_from_ways), len(shpobjs_from_rltns))
    shpobjs = []
    shpobjs.extend(shpobjs_from_nodes)
    shpobjs.extend(shpobjs_from_ways)
    shpobjs.extend(shpobjs_from_rltns)
    gpdf = gp.GeoDataFrame(shpobjs, columns=columns)
    return gpdf


In [7]:
print 'begin OSM POI pipeline'
data_dir = '../data/'
osm_data_dir = data_dir + 'osm/'

# 1. read OSM data
start_time = datetime.datetime.now()
path_osm_dc_bbox = osm_data_dir + 'osm_dc_bbox.osm'
osm_dc_bbox = OSMContainer(path_osm_dc_bbox)
print 'finished reading osm data'

# 2. retrieve osm within dc polygon - from/data_prepro/getOSM_within_dc_poly
print '2.'
start_time = datetime.datetime.now()
path_osm_ids_in_dc = osm_data_dir + 'osm_ids_within_dc_polygon.txt'
with open(path_osm_ids_in_dc) as f:
    osm_ids_in_dc = eval(f.readlines()[0])
dc_nodes = filter_objs_by_ids(osm_dc_bbox.osm_objs['Node'], osm_ids_in_dc['Node'])
dc_ways = filter_objs_by_ids(osm_dc_bbox.osm_objs['Way'], osm_ids_in_dc['Way'])
dc_rltns = filter_objs_by_ids(osm_dc_bbox.osm_objs['Relation'], osm_ids_in_dc['Relation'])
print 'retrieve osm within dc polygon'

begin OSM POI pipeline
begin reading osm 2016-11-12 21:41:07.818000
finish reading osm 2016-11-12 21:42:21.408000
finished reading osm data
2.
retrieve osm within dc polygon


In [8]:
dc_nodes_with_tag = filter_osm_data(dc_nodes)
dc_ways_with_tag = filter_osm_data(dc_ways)
dc_rltns_with_tag = filter_osm_data(dc_rltns)
print 'len with tags: node={}, way={}, relation={}'.format(
    len(dc_nodes_with_tag), len(dc_ways_with_tag), len(dc_rltns_with_tag))


len with tags: node=37912, way=190931, relation=2552


In [9]:

path_opendc_segs = data_dir + 'opendc_segments.geojson'
segs = gp.read_file(path_opendc_segs)

In [13]:

def flatten_rltn_shpcltn(shpcltn):
    flat_shpcltn = []
    for shpobjs in shpcltn:
        flat_shpcltn.extend(shpobjs)
    return flat_shpcltn

def build_gpdf_list_from_osm(osm_data, nodes, ways, rltns):
    columns = ['id', 'type', 'geometry']
    shpobjs_from_nodes = [(node.id, 'Node', node2pt(node)) for node in nodes]
    shpobjs_from_ways = [(way.id, 'Way', way2lineOrpoly(osm_data, way)) for way in ways]
    shpobjs_from_rltns = [(rltn.id,rltn2mergedFlattenListShp(osm_data,rltn)) for rltn in rltns]
    shpobjs_from_rltns = [r for r in shpobjs_from_rltns if r[1]]
    shpobjs_from_rltns = [(rid, 'Relation', shpobj) for (rid,shpobjs) in shpobjs_from_rltns for shpobj in shpobjs]
    print 'build_gpdf_list len(shpobjs_from_nodes)={},len(shpobjs_from_ways)={},len(shpobjs_from_rltns)={}'.format(
        len(shpobjs_from_nodes), len(shpobjs_from_ways), len(shpobjs_from_rltns))
    shpobjs = []
    shpobjs.extend(shpobjs_from_nodes)
    shpobjs.extend(shpobjs_from_ways)
    shpobjs.extend(shpobjs_from_rltns)
    gpdf = gp.GeoDataFrame(shpobjs, columns=columns)
    return gpdf
osm_gpdf = build_gpdf_list_from_osm(osm_dc_bbox,dc_nodes_with_tag,dc_ways_with_tag, dc_rltns_with_tag)


Ring Self-intersection at or near point -77.020059900000007 38.815362499999999
D:\���ѧϰ����\Life in Maryland\Research Project��CyclingSafe\github\Cycling_Safe\utils\osm\osm2shp.py:30: UserWarning: way id=59249848 is not valid as a <class 'shapely.geometry.polygon.Polygon'>
  warnings.warn("way id={} is not valid as a {}".format(way.id, type(obj)))
Self-intersection at or near point -77.047313000000003 38.907462299999999
D:\���ѧϰ����\Life in Maryland\Research Project��CyclingSafe\github\Cycling_Safe\utils\osm\osm2shp.py:30: UserWarning: way id=66646317 is not valid as a <class 'shapely.geometry.polygon.Polygon'>
  warnings.warn("way id={} is not valid as a {}".format(way.id, type(obj)))
Self-intersection at or near point -77.018634802813295 38.911758200000001
D:\���ѧϰ����\Life in Maryland\Research Project��CyclingSafe\github\Cycling_Safe\utils\osm\osm2shp.py:30: UserWarning: way id=67232331 is not valid as a <class 'shapely.geometry.polygon.Polygon'>
  warnings.warn("way id={} is not va

build_gpdf_list len(shpobjs_from_nodes)=37912,len(shpobjs_from_ways)=190931,len(shpobjs_from_rltns)=3174


D:\���ѧϰ����\Life in Maryland\Research Project��CyclingSafe\github\Cycling_Safe\utils\osm\osm2shp.py:30: UserWarning: way id=297747861 is not valid as a <class 'shapely.geometry.polygon.Polygon'>
  warnings.warn("way id={} is not valid as a {}".format(way.id, type(obj)))


In [ ]:

osm_ids_near = get_osm_ids_near_dc_seg(segs, osm_gpdf, bfr_20m)


seg nearby.py: get_osm_ids_near_dc_seg, len duplicated ids = 19386 len unique ids= 19386
seg nearby.py: get_osm_ids_near_dc_seg, len duplicated ids = 2626 len unique ids= 2203
seg nearby.py: get_osm_ids_near_dc_seg, len duplicated ids = 140629 len unique ids= 140629


In [ ]:
osm_ids_for_merge = osm_ids_near
# filter osm for merge
dc_nodes_for_merge = filter_objs_by_ids(dc_nodes, osm_ids_for_merge['Node'])
dc_ways_for_merge = filter_objs_by_ids(dc_ways, osm_ids_for_merge['Way'])
dc_rltns_for_merge = filter_objs_by_ids(dc_rltns, osm_ids_for_merge['Relation'])
print 'filter by id len ways = {}, relations = {}'.format(len(dc_ways_for_merge), len(dc_rltns_for_merge))

filter_for_ways = [filter_isnot_admin, filter_isnot_motorway, filter_isnot_landuse]
dc_ways_for_merge = filter_osm_data(dc_ways_for_merge, special_filters=filter_for_ways)
filter_for_rltns = [filter_isnot_admin, filter_isnot_restriction, filter_isnot_landuse]
dc_rltns_for_merge = filter_osm_data(dc_rltns_for_merge, special_filters=filter_for_rltns)
print 'filter by tag len ways = {}, relations = {}'.format(len(dc_ways_for_merge), len(dc_rltns_for_merge))

## do merge on filtered by ids
gpdf_for_merge = build_gpdf_list_from_osm(osm_dc_bbox, dc_nodes_for_merge, dc_ways_for_merge, dc_rltns_for_merge)
_, all_parents, direct_parent, osm_ids_equal_pair = merge_within(gpdf_for_merge)


filter by id len ways = 140629, relations = 2203
filter by tag len ways = 127877, relations = 1394

Ring Self-intersection at or near point -77.020059900000007 38.815362499999999
Self-intersection at or near point -77.047313000000003 38.907462299999999
Self-intersection at or near point -77.018634802813295 38.911758200000001
Self-intersection at or near point -77.021721999999997 38.919942800000001
Self-intersection at or near point -76.960937168640712 38.892445184048071
